# Lab Risk management

## Imports

For the later we will import the usual libraries :

In [ ]:

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import proj3d


In order to have 3D interractive visualization, it is necessary to activate the widget mode of matplotlib

In [ ]:
%matplotlib widget

If ipympl is missing, you can install ipympl with the following command line (uncomment first). Be aware that a pip install can broke your python configuration. Use it if you are familiar with such command.

In [ ]:
#!pip install ipympl

## Lab 1 : PCA sandbox

The objectiv of this lab is to study the impact of correlated random variable throug the analysis of its principal components

### Dataset creation

This first step intends to create a set of 3 correlated random variables. We define the number of point to draw, the average and volatility for each random variable.

In [ ]:
num_samples = 400

mu = np.array([5.0, 3.0, 10.0])
sigma = np.array([5.0, 3.0, 1.0])

As we intends to have corralated matrix, we will use the following correlation matrix :

In [ ]:
correlation_matrix = np.array([
        [  1.00, -0.64, -0.97],
        [ -0.64,  1.00,  0.57],
        [ -0.97,  0.57,  1.00]
    ])

You can visualise the object by directly input its name in a code cell:
    

In [ ]:
correlation_matrix

You can also use print:

In [ ]:
print(correlation_matrix)

We will use a generator of correlated variable that require a covariance matrix. Compute the covariance matrix:

In [ ]:
covariance_matrix = correlation_matrix * np.outer(sigma,sigma)
covariance_matrix

Once we have a covariance matrix, we can ask for correlated random variables that follow a normal distribution:

In [ ]:
# Generate the random samples.
rng = np.random.default_rng()
dataset = rng.multivariate_normal(mu, covariance_matrix, size=num_samples)

Lets inspect the output:

In [ ]:
dataset

In order to have the dimension of dataset, we can use shape function : 

In [ ]:
dataset.shape

Lets inspect the first element of the dataset variable:

In [ ]:
dataset[0]

If we want the first random variable, we need to ask for all value on the first axis, and ask for the first element on the second variable:

In [ ]:
dataset[:,0]

If we only want to retreive the n first elements of the first random variable, we can use the slice operator (":") with a precision of boundaries:

In [ ]:
print(dataset[0:10,0])

Lets store in x1, y1 and z1 the tree random variables:

In [ ]:
x1 = dataset[:,0]
y1 = dataset[:,1]
z1 = dataset[:,2]

Lets plots those variables:

In [ ]:
# Plot various projections of the samples.
plt.subplot(2,2,1)
plt.plot(x1, y1, 'b.', alpha=0.25)
plt.plot(x1.mean(), y1.mean(), 'ro', ms=3.5)
plt.ylabel('y')
plt.axis('equal')
plt.grid(True)

plt.subplot(2,2,3)
plt.plot(x1, z1, 'b.', alpha=0.25)
plt.plot(x1.mean(), z1.mean(), 'ro', ms=3.5)
plt.xlabel('x')
plt.ylabel('z')
plt.axis('equal')
plt.grid(True)

plt.subplot(2,2,4)
plt.plot(y1, z1, 'b.', alpha=0.25)
plt.plot(y1.mean(), z1.mean(), 'ro', ms=3.5)
plt.xlabel('y')
plt.axis('equal')
plt.grid(True)

plt.show()

And in 3D:

In [ ]:
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection='3d')

ax.scatter(x1, y1, z1)
plt.show()

Lets see if our dataset has the correct properties. Compute the average and standard deviation:

In [ ]:
empirical_average = dataset.mean(axis=0)
empirical_average

In [ ]:
empirical_average = dataset.std(axis=0)
empirical_average

Now we will normalize our dataset (average = 0 and standard deviation = 1):

In [ ]:
normalised_dataset = (dataset - dataset.mean(axis=0))/dataset.std(axis=0)
normalised_dataset

Lets verify that our dataset still has the same look:

In [ ]:
x2, y2, z2 = normalised_dataset[:,0], normalised_dataset[:,1], normalised_dataset[:,2]


fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection='3d')

ax.scatter(x2, y2, z2)
plt.show()

Now compute the correlation matrix of the normalized dataset:

In [ ]:
noramlized_correlation_matrix = np.cov(normalised_dataset, rowvar=False)
noramlized_correlation_matrix

Then we compute the eigen values and eigen vectors of this normalized correlation matrix. Look in Numpy documentation:

In [ ]:
eigen_values, eigen_vectors = np.linalg.eig(noramlized_correlation_matrix)

Lets inspect eigen values:

In [ ]:
eigen_values

In [ ]:
100*eigen_values/sum(eigen_values)

In [ ]:
np.sum(eigen_values)

In [ ]:
eigen_vectors

For ease of usage, we will sort eigen values and vectors by order of magnitude of aigen values:

In [ ]:
sorted_index=np.argsort(-eigen_values)
sorted_eigen_values = eigen_values[sorted_index]
sorted_eigen_vectors = eigen_vectors[:,sorted_index]

In [ ]:
sorted_eigen_values

In [ ]:
sorted_eigen_vectors

Now we choose the number of components:

In [ ]:
nb_components = 3


Select only the first nb_components:

In [ ]:
sorted_eigen_vectors[:,:nb_components]

To compute the principal components, multiply the normalised_dataset to the sorted_eigen_vectors. Use nb_components as a filter of number of components.

In [ ]:
principal_components = np.matmul(normalised_dataset,sorted_eigen_vectors[:,:nb_components])

Lets check some properties on the principal components:

In [ ]:
print(principal_components.mean(axis=0))

In [ ]:
np.cov(principal_components,rowvar=False)

In [ ]:
#np.cov(principal_components,rowvar=False).trace()
print(np.sum(principal_components.std(axis=0)*principal_components.std(axis=0)))

No lets reconstruct the original dataset only using the selected components:

In [ ]:
reconstructed_dataset= np.matmul(principal_components,np.transpose(sorted_eigen_vectors[:,:nb_components]))

As we have previously normilzed our dataset, lets denormlized the dataset:

In [ ]:
reconstructed_dataset_denormalized = reconstructed_dataset * dataset.std(axis=0) + dataset.mean(axis=0)

In [ ]:
reconstructed_dataset_denormalized

In [ ]:
dataset

Lets see the differences between datasets:

In [ ]:
reconstructed_dataset_denormalized - dataset

In [ ]:
print(np.average(abs(reconstructed_dataset_denormalized - dataset)))

And plot the reconstructed dataset

In [ ]:
x3, y3, z3 = reconstructed_dataset_denormalized[:,0], reconstructed_dataset_denormalized[:,1], reconstructed_dataset_denormalized[:,2]


fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection='3d')

ax.scatter(x3, y3, z3)
plt.show()

Last try to modify number of components and observe the effect on the reconstructed dataset

## Lab 2: PCA on equity

We will work on 7 equity index that we will retreive on yahoo fincial:

In [ ]:
yahoo_url = 'https://query1.finance.yahoo.com/v7/finance/download/{}?period1=1509235200&period2=1667001600&interval1&events=history'
tickers = ['^GSPC','^FCHI','^FTSE','^NDX','^STOXX50E','^DJI','^IBEX']

In [ ]:
df_raw = pd.DataFrame()
for ticker in tickers:
    url = yahoo_url.format(ticker)
    df_tmp = pd.read_csv(url)
    df_tmp['Ticker'] = ticker
    df_raw = pd.concat([df_raw, df_tmp[['Date','Ticker','Adj Close']]])
df_raw.columns = ['date','ticker','price']
df_raw

In [ ]:
df_equity = df_raw.pivot_table(index=['date'], columns='ticker', values=['price'])
df_equity

Plot the equities time series directly using pandas:

In [ ]:
df_equity.plot()

Now, again using pandas, compute daly returns:

In [ ]:
df_returns = df_equity["price"].pct_change()[1:]
df_returns

And plot the returns:

In [ ]:
df_returns.plot()

Observe the histogram of returns:

In [ ]:
df_returns.hist()

Now, compute the cumulated sum of historical returns:

In [ ]:
df_cumulated_returns = pd.DataFrame(df_returns.to_numpy().cumsum(axis=0))
df_cumulated_returns.columns = df_returns.columns
df_cumulated_returns.plot()

In [ ]:
#df_returns.to_excel("df_equity.xlsx")

Check the average return and associated volatility:

In [ ]:
df_returns.mean()

In [ ]:
df_returns.std()

Now compute the normalized returns:

In [ ]:
df_returns_normalized =  (df_returns - df_returns.mean())/df_returns.std()

In [ ]:
df_returns_normalized.plot()

And average return and volatility:

In [ ]:
df_returns_normalized.mean()

In [ ]:
df_returns_normalized.std()

Compute the correlation matrix:

In [ ]:
equity_correlation_matrix = df_returns_normalized.corr()

In [ ]:
equity_correlation_matrix

Lets plot it with seaborn:

In [ ]:
with plt.ioff():
    fig = plt.figure()
    sns.heatmap(equity_correlation_matrix,
            xticklabels=equity_correlation_matrix.columns,
            yticklabels=equity_correlation_matrix.columns)
    display(fig)
    


Now we will extract time series from dataframe to numpy array:

In [ ]:
equity_correlation_matrix_np = equity_correlation_matrix.to_numpy()
equity_correlation_matrix_np

Computes the eigen values and vectors:

In [ ]:
equity_eigen_values, equity_eigen_vectors = np.linalg.eig(equity_correlation_matrix_np)
equity_sorted_index = np.argsort(-equity_eigen_values)
equity_eigen_values_sorted = equity_eigen_values[equity_sorted_index]
equity_eigen_vectors_sorted = equity_eigen_vectors[equity_sorted_index]

In [ ]:
100*equity_eigen_values_sorted/equity_eigen_values_sorted.sum()

In [ ]:
explained_variance = [100*equity_eigen_values_sorted[i]/equity_eigen_values_sorted.sum() for i in range(len(equity_eigen_values_sorted))]
explained_variance
fig = plt.figure()
plt.bar(range(0,len(explained_variance)),explained_variance)
plt.show()

Compute the principal components:

In [ ]:
equity_principal_components = np.matmul(df_returns_normalized.to_numpy(),equity_eigen_vectors_sorted)

In [ ]:
equity_nb_comp = 7

Reconstruct the returns only using equity_nb_comp:

In [ ]:
new_equity_returns = np.matmul(equity_principal_components[:,:equity_nb_comp], np.transpose(equity_eigen_vectors_sorted[:,:equity_nb_comp])) * df_returns.std().to_numpy() + df_returns.mean().to_numpy()

In [ ]:
df_new_returns = pd.DataFrame(np.cumsum(new_equity_returns,axis=0))

In [ ]:
df_new_returns.plot()

In [ ]:
df_cumulated_returns.plot()

Mesure the error:

In [ ]:
abs(df_returns-new_equity_returns).sum().sum()

In [ ]:
outTab = []

for nb in range(1,len(df_returns.columns)+1):
    temp_equity_returns = np.matmul(equity_principal_components[:,:nb], np.transpose(equity_eigen_vectors_sorted[:,:nb])) * df_returns.std().to_numpy() + df_returns.mean().to_numpy()
    outTab.append([nb,abs(df_returns-temp_equity_returns).sum().sum()])

In [ ]:
outTab

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=7)
out_pca = pca.fit_transform(df_returns_normalized)

fig = plt.figure()
plt.bar(range(0,len(pca.explained_variance_)),pca.explained_variance_)
plt.show()

# Lab 3: PCA on yield curve

In [ ]:
#pd.read_csv("https://www.federalreserve.gov/data/yield-curve-tables/feds200628.csv")

try:
    from urllib.request import Request, urlopen  # Python 3
except ImportError:
    from urllib2 import Request, urlopen  # Python 2

req = Request("https://www.federalreserve.gov/data/yield-curve-tables/feds200628.csv")
req.add_header('User-Agent', 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:77.0) Gecko/20100101 Firefox/77.0')
content = urlopen(req)

df = pd.read_csv(content,skiprows=9)
df = df.set_index("Date")
for column in df.columns:
    if column[:5] != "SVENY":
        df=df.drop(column,axis=1)
df.plot()


In [ ]:
df = df.fillna(method='ffill')

In [ ]:
df.plot()